In [15]:
# import packages 
import requests
import pandas as pd

# `需求`：
爬取国家药品监督管理总局基于中华人民共和国化妆品生产许可证相关数据

`url`：http://scxk.nmpa.gov.cn:81/xk/


In [9]:
url = 'http://scxk.nmpa.gov.cn:81/xk/'

In [11]:
# UA 伪装
headers = {
    'User-Agent':
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'
}

In [14]:
#写入邮件
page_text = requests.get(url=url, headers=headers).text
with open('./huazhuangpin.html','w',encoding= 'utf-8')as fp:
    fp.write(page_text)

---
# 爬取第一页数据

**动态加载数据**：
首页中动态加载数据是通过ajax动态请求得到的
    - 通过对详情页url的观察，url的域名都一样，只有携带的参数（id）不一样
    - id值可以从首页对应的ajax请求的json串中获取
    - 域名和id值拼接出一个完整的企业对应的url

详情页数据也是通过ajax数据包动态加载出来的

步骤：
主页爬取id - 利用id爬取详情页数据

http://scxk.nmpa.gov.cn:81/xk/itownet/portal/dzpz.jsp?id=103caf86662d441d810dacc7dbb109a8
http://scxk.nmpa.gov.cn:81/xk/itownet/portal/dzpz.jsp?id=ff83aff95c5541cdab5ca6e847514f88

In [86]:
# url
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'

# 参数封装
data = {
'on': 'true',
'page': '1',
'pageSize': '15',
'productName':'',
'conditionType': '1',
'applyname':'',
'applysn':''
}

# UA 伪装
headers = {
    'User-Agent':
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'
}

# 发起post
r = requests.post(url =url, headers = headers,data = data).json()

In [87]:
id_list = []
for dic in r['list']:
    id_list.append(dic['ID'])
print('---done---')

---done---


In [88]:
# 获取企业详细数据
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById'

#参数封装
all_data_list = []
for company_id in id_list:
    data = {'id': company_id}
    #发起请求
    all_data_list.append(requests.post(url,data = data , headers = headers).json())
print('----done---')

----done---


In [89]:
page1 = pd.DataFrame(all_data_list)
page1.head(3)

,businessLicenseNumber,businessPerson,certStr,cityCode,countyCode,creatUser,createTime,endTime,epsAddress,epsName,...,rcManagerDepartName,rcManagerUser,startTime,xkCompleteDate,xkDate,xkDateStr,xkName,xkProject,xkRemark,xkType
0,91440101MA5CYUF0XX,陈飞,一般液态单元（护发清洁类、护肤水类、啫喱类）；膏霜乳液单元（护肤清洁类、护发类）,,,,,,广州市白云区均禾街清湖大塘大道11号（自主申报）,广东天姿化妆品科技有限公司,...,白云区局均禾市场监管所,监管机构指定人员,,None,2025-01-16,2021-05-17,严振,,无,201
1,91350603MA2YM6P4X2,李周欣,一般液态单元（护发清洁类、护肤水类）；膏霜乳液单元（护发类、护肤清洁类）；粉单元（散粉类、浴...,,,,,,漳州市蓝田经济开发区,青蛙王子（福建）婴童护理用品有限公司,...,漳州市龙文区市场监督管理局蓝田市场监管所,吴少舟、许志宏,,None,2023-04-15,2020-09-08,郑纯,,,206
2,9135072459788424XQ,徐文胜,粉单元（块状粉类、散粉类）；蜡基单元（蜡基类）,,,,,,松溪县经济技术开发区,福建金亿文化用品有限公司,...,南平市松溪县市场监督管理局,林银良；陈芬丽；陈淑青,,None,2022-01-05,2020-09-08,郑纯,,,206


---
# 爬取多个页面数据

In [105]:
# url
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'

# UA 伪装
headers = {
    'User-Agent':
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.2 Safari/605.1.15'
}

# 参数封装
id_list = []
for page in range(1, 4):
    page = str(page)
    data = {
        'on': 'true',
        'page': page,
        'pageSize': '15',
        'productName': '',
        'conditionType': '1',
        'applyname': '',
        'applysn': ''
    }
    for dic in requests.post(url=url, headers=headers, data=data).json()['list']:
        id_list.append(dic['ID'])
    print('=====done====')

=====done====
=====done====
=====done====


In [108]:
print(len(id_list))

45


In [ ]:
# 获取企业详细数据
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsById'

#参数封装
all_data_list = []
for company_id in id_list:
    data = {'id': company_id}
    #发起请求
    all_data_list.append(requests.post(url,data = data , headers = headers).json())
print('----done---')

In [ ]:
page3 = pd.DataFrame(all_data_list)
page3